# Compute the observation baseline 

The notebook allows to compute the observation baseline from X data (ground station observations, train part). 
The observation baseline is the accumulated rainfall on 24 hours the day before. 
Here are the steps :
- compute the accumulated rainfall per day (the X data have a 1 hour time frequency)
- select the observations the day before 
- create the ID column : id_station + month + index day value

In [1]:
import pandas as pd
import datetime
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [4]:
path = '/home/douzery/Bureau/Defi-IA-2022_Data/Train/X_station_train.csv'
first_date = datetime.datetime(2016,1,1)    
last_date = datetime.datetime(2017,12,31)
output_file = "my_baseline_obs.csv"   #if you want to save your baseline in a csv file 

In [5]:
#read the ground station data
def read_gs_data(fname):
    gs_data = pd.read_csv(fname,parse_dates=['date'],infer_datetime_format=True)
    gs_data = gs_data.sort_values(by=["number_sta","date"])
    return gs_data

x = read_gs_data(path)
x['number_sta']=x['number_sta'].astype('category')

#sort by station, then by date 
x = x.sort_values(['number_sta','date'])
x

,number_sta,date,ff,t,td,hu,dd,precip,Id
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4
...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22


In [9]:
#get the observation baseline
Base_obs = x[{"number_sta","date","precip"}]
Base_obs.set_index('date',inplace = True)  

#compute the accumulated rainfall per day with nan management 
#if any NaN on the day, then the value is NaN (24 values per day)
Base_obs = Base_obs.groupby('number_sta').resample('D').agg(pd.Series.sum, min_count = 24)
Base_obs = Base_obs.reset_index(['date','number_sta'])
Base_obs['number_sta'] = Base_obs['number_sta'].astype('category') 
Base_obs

,number_sta,date,precip
0,14066001,2016-01-01,0.2
1,14066001,2016-01-02,3.4
2,14066001,2016-01-03,11.7
3,14066001,2016-01-04,0.6
4,14066001,2016-01-05,0.4
...,...,...,...
183742,95690001,2017-12-26,2.4
183743,95690001,2017-12-27,3.2
183744,95690001,2017-12-28,0.0
183745,95690001,2017-12-29,4.4


In [10]:
#Select the observations the day before 
Base_obs['baseline_obs'] = Base_obs.groupby(['number_sta'])['precip'].shift(1)
Base_obs = Base_obs.sort_values(by=["number_sta","date"])
del Base_obs['precip']
Base_obs = Base_obs.rename(columns={'baseline_obs':'precip'})
Base_obs

,number_sta,date,precip
0,14066001,2016-01-01,NaN
1,14066001,2016-01-02,0.2
2,14066001,2016-01-03,3.4
3,14066001,2016-01-04,11.7
4,14066001,2016-01-05,0.6
...,...,...,...
183742,95690001,2017-12-26,0.0
183743,95690001,2017-12-27,2.4
183744,95690001,2017-12-28,3.2
183745,95690001,2017-12-29,0.0


In [11]:
#get the day indexes (to the final Id)
date = first_date
dates = []
while date <= (last_date - datetime.timedelta(days=1)):
    date += datetime.timedelta(days=1)
    dates.append(date)

d_dates =  pd.DataFrame(dates, columns = ['date'])
d_dates['day_index'] = d_dates.index
d_dates

,date,day_index
0,2016-01-02,0
1,2016-01-03,1
2,2016-01-04,2
3,2016-01-05,3
4,2016-01-06,4
...,...,...
725,2017-12-27,725
726,2017-12-28,726
727,2017-12-29,727
728,2017-12-30,728


In [12]:
#create the ID column (id_station + month + index value)
y_f = pd.merge(Base_obs,d_dates,how="right",on = ["date"])
y_f = y_f[y_f['date']!=last_date]
y_f['Id'] = y_f['number_sta'].astype(str) + '_' + \
                  y_f['day_index'].astype(str) 
y_f

,number_sta,date,precip,day_index,Id
0,14066001,2016-01-02,0.2,0,14066001_0
1,14126001,2016-01-02,0.5,0,14126001_0
2,14137001,2016-01-02,NaN,0,14137001_0
3,14216001,2016-01-02,0.6,0,14216001_0
4,14296001,2016-01-02,0.2,0,14296001_0
...,...,...,...,...,...
183493,86137003,2017-12-30,1.2,728,86137003_728
183494,86165005,2017-12-30,1.2,728,86165005_728
183495,86272002,2017-12-30,1.6,728,86272002_728
183496,91200002,2017-12-30,7.3,728,91200002_728


In [13]:
#final post-processing
del y_f['day_index']
y_f = y_f.rename(columns={'precip':'Prediction'})
y_f

,number_sta,date,Prediction,Id
0,14066001,2016-01-02,0.2,14066001_0
1,14126001,2016-01-02,0.5,14126001_0
2,14137001,2016-01-02,NaN,14137001_0
3,14216001,2016-01-02,0.6,14216001_0
4,14296001,2016-01-02,0.2,14296001_0
...,...,...,...,...
183493,86137003,2017-12-30,1.2,86137003_728
183494,86165005,2017-12-30,1.2,86165005_728
183495,86272002,2017-12-30,1.6,86272002_728
183496,91200002,2017-12-30,7.3,91200002_728


In [9]:
#to save the baseline in a csv file 
#y_f.to_csv('/kaggle/working/' + output_file,index=False)